In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.linear_model import LinearRegression
from math import sqrt
import warnings
warnings.filterwarnings("ignore")

from env import user, host, password 
import wrangle
import split_scale
import features

In [98]:
import evaluateORIG

##### As a customer analyst, I want to know who has spent the most money with us over their lifetime. I have monthly charges and tenure, so I think I will be able to use those two attributes as features to estimate total_charges. I need to do this within an average of $5.00 per customer.

## select your features using your features script


In [3]:
df = wrangle.wrangle_telco()

In [ ]:
df.info()

In [4]:
train,test=split_scale.split_my_data(df)

In [5]:
train = train.set_index('customer_id') 
test = test.set_index('customer_id')

In [6]:
scaler,train_scaled, test_scaled =split_scale.standard_scaler(train,test)

In [7]:
X_train=train_scaled.drop(columns=['total_charges'])
X_test=test_scaled.drop(columns=['total_charges'])
y_train=train[['total_charges']]
y_test=test[['total_charges']]

In [8]:
X_train.head()

,monthly_charges,tenure
2192-CKRLV,-0.349191,0.839851
7596-IIWYC,-1.184789,-1.700684
6408-WHTEF,0.811120,0.839851
3070-FNFZQ,-1.180459,-1.926509
8774-GSBUN,-1.036142,0.839851


In [9]:
y_train.head()

,total_charges
customer_id,
2192-CKRLV,3580.95
7596-IIWYC,538.20
6408-WHTEF,6376.55
3070-FNFZQ,478.75
8774-GSBUN,1797.10


In [10]:
X_test.head()

,monthly_charges,tenure
5248-KWLAR,0.838541,0.783394
7306-YDSOI,-0.044682,0.670482
8454-AATJP,1.551469,0.839851
7552-KEYGT,-1.204993,-1.700684
9479-HYNYL,0.560008,0.783394


In [11]:
y_test.head()

,total_charges
customer_id,
5248-KWLAR,6325.25
7306-YDSOI,4069.90
8454-AATJP,8405.00
7552-KEYGT,520.55
9479-HYNYL,5705.05


In [12]:
optimal_n_features = features.optimal_number_of_features(X_train, y_train, X_test, y_test)

In [13]:
selected_features=features.optimal_features(X_train,y_train,optimal_n_features)

In [14]:
#Looks like tenure is not that important
selected_features

Index(['monthly_charges'], dtype='object')

## START THE SKLEARN MODEL 3-STEP 
    #####1 Call down the model from the SKLEARN world of platonic forms
    #####2 Fit the training data to the model (X and Y)
    #####3 Make some predictions or transformations with this fitted model

In [15]:
#INVOKE THE MODEL
lin_model_1=LinearRegression()

In [16]:
#Fit that model to the training data
lin_model_1.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [17]:
#Looks like there is a stronger 
lin_model_1_y_intercept=lin_model_1.intercept_
print("intercept: ",lin_model_1_y_intercept)
lin_model_1_coefficents=lin_model_1.coef_
print("coefficents: ",lin_model_1_coefficents)

intercept:  [3759.66194362]
coefficents:  [[2114.82024128  748.30645931]]


In [111]:
y_predicted=lin_model_1.predict(X_test)

In [ ]:
type(y_predicted)

In [19]:
mse_lm1=mean_squared_error(y_test,y_predicted)
print("linear model \n mean_squared error: {:.3}".format(mse_lm1))

r2_lm1=r2_score(y_test,y_predicted)
print('{:.2%} of the variance in the student''s final grade can be explained by monthly charges and tenure'.format(r2_lm1))


linear model 
 mean_squared error: 2.26e+05
96.55% of the variance in the students final grade can be explained by monthly charges and tenure


## The evaluate functions are crapping out on me. I'm having to do them by hand

In [23]:
yhat = y_train.mean()

In [24]:
n = y_train.size

In [26]:
residuals = yhat - y_train

In [56]:
X_train.dtypes

monthly_charges    float64
tenure             float64
dtype: object

In [46]:
base_sse = (residuals**2).sum()

In [47]:
base_mse = sse/y_train.size

### Got the baseline sse and mse, now to make the one with the model we fit

In [100]:
y_test.shape

(337, 1)

In [50]:
y_predicted.shape

(337, 1)

In [ ]:
#When I do it line by line inside the notebook it works. When I call my function or the one from
#the Bayes repository. Oh well...

In [72]:
residual = y_test - y_predicted

In [74]:
residual2 = residual ** 2

In [76]:
sse = residual2.sum()

### THAT IS THE SSE FOR THE LINEAR MODEL THAT ONLY USES 'monthly charges' as a predictor

In [77]:
sse

total_charges    7.632944e+07
dtype: float64

In [87]:
y_mean = y_train.mean()

In [94]:
y_mean.values

array([3759.66194362])

In [113]:
mse = sum((y_predicted - y_mean.values)**2)

In [103]:
y_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 337 entries, 5248-KWLAR to 5884-FBCTL
Data columns (total 1 columns):
total_charges    337 non-null float64
dtypes: float64(1)
memory usage: 5.3+ KB


In [107]:
#y_predicted = pd.DataFrame(y_predicted)

In [108]:
y_predicted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 337 entries, 0 to 336
Data columns (total 1 columns):
0    337 non-null float64
dtypes: float64(1)
memory usage: 2.7 KB


In [128]:
#evaluateORIG.regression_errors(y_test, y_predicted)

In [112]:
y_test - y_predicted

,total_charges
customer_id,
5248-KWLAR,206.006085
7306-YDSOI,-96.994053
8454-AATJP,735.795140
7552-KEYGT,581.865302
9479-HYNYL,174.852366
1951-IEYXM,265.099187
2996-XAUVF,-206.881698
8237-ULIXL,-291.450768
3910-MRQOY,-334.176962


In [114]:
#blob = evaluate.baseline_mean_errors(y_train)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [119]:
#blobORIG = evaluateORIG.baseline_mean_errors(y_train)

In [118]:
y_train.mean()

total_charges    3759.661944
dtype: float64

In [ ]:
pd.DataFrame({'actual': y_unscaled,
             'lm1': y_lm1_unscaled.ravel(),
             'lm_baseline':y_baseline_unscaled.ravel()})\
.melt(id_vars=['actual'],var_name='model',value_name='prediction')\
.pipe((sns.relplot,'data'),x='actual',y='prediction',hue='model')
min, max = 60, 100
plt.plot([min, max], [min, max], c='black', ls=':')
plt.ylim(min, max)
plt.xlim(min, max)
plt.title('Predicted vs Actual Final Grade')